웹 크롤링
=========

*how to?*
---------

> ### **1.필요한 모듈 불러오기**
>>     from bs4 import BeautifulSoup , import requests 
> ### **2. html 구조 이해하기**
>> * mac-> 개발자 도구 활성화 - 페이지 소스 보기/ 윈도우 - F12
>> * <여는태그 속성명="속성값">내용</닫는태그>
>> * <태그 class = "클래스명" > 
>>> ex) div : 구역 표시하는 태그, p : 문단 표시하는 태그, body : 실제적으로 보여지는 부분 나타내는 태그...

> ### **3.텍스트 잘 뽑아오기**
>>    위치 정확히 지정, 클래스나 태그 명 잘 보기.
>>>     find(id='id명')
    findAll('tag') or find_all('tag)
    findAll('tag', {class:'class'})
    find('class')
    select('위계구조')
    select_one('원하는 것')
    
>>>>**Select 와 find의 차이점**
>>>>>select finds multiple instances and returns a list, find finds the first, so they don't do the same thing. select_one would be the equivalent to find.
    
>>>> 
    get_text()
    text()
    next_sibling : 텍스트 노드인 경우



### 1. IDMB 영화 리뷰 크롤링

In [1]:
# 1. 웹크롤링에 필요한 2가지 모듈 호출
from urllib.request import urlopen # 특정 웹서버에 접근
from bs4 import BeautifulSoup # 웹페이지 내용구조 해석
import requests #파이썬에서 http 요청 보내는 라이브러리 
from urllib import parse
from bs4 import BeautifulSoup #html 구조에서 간딴히 정보 빼내기 위해 사용하는 라이브러리

#url 지정
url= 'https://www.imdb.com/title/tt4154756/reviews?ref_=tt_ql_3'

#webpage에 urlopen으로 열어줘서 할당
res = requests.get(url)


#source 파일에....
source = BeautifulSoup(res.text, 'html.parser')

In [3]:
# source

In [4]:
# review_list = source.find_all('div', {'class': 'imdb-user-review'})
# review_p = source.p.string
# review_list2 = source.find_all(['a','span'])
review_only =  source.find_all('div',{'class': "text show-more__control"})

In [9]:
review_only[3]

<div class="text show-more__control">A wonderful movie and the action scenes in the film are some of the most wonderful, especially in the Battle of Titan. I liked the fact that Thanos was capable of defeating all the battles in the film and he did not use a plan and the end was sad, especially the death of Spider-man</div>

In [11]:
#화면 속 글자만 가져오고 싶다!!
# for review in review_only:
#        print(review.get_text().strip())

In [15]:
# lis

In [16]:
lis = source.select('div.lister-list > div.imdb-user-review')

for i in range(len(lis)):
    score = lis[i].find('span').text.strip()[0]
    text = lis[i].find('div',{'class':'text show-more__control'}).text.strip()
    print('score :' + score)
    print('    ')
    print('comment: ' + text)
    print('    ')

score :9
    
comment: This movie portrays a villain that by far has been the best out of all the other Marvel antagonists. It shows his personal conflicts, his purpose, as well as his cruelty. The heroes of this film are overshadowed by this, but the ending really makes a mark on the audience.
    
score :1
    
comment: This film is way better than endgame!
The actions better
The writings better
The dialogues better
The cgi is better
More shocks and more emotional.
Watching both in the cinema , the reactions to this in cinema was unbelievable compared to endgame. Dont get me wrong endgame is good but to say its the best marvel film is jus marvel fan boys being stupid.
I just dont see how people are saying endgame is the best marvel film when theres clearly other marvel films better than it!
But trust me this is the film that changed the mcu , lol maybe it because i wanted them to lose because thanos is badass!!!!!!!!!!!!!!
    
score :1
    
comment: I consider myself a big comic boo

## *코로나 현황 크롤링 실습* 
간단 ver.

In [17]:
# 오늘의 코로나 현황

webpage = urlopen('http://ncov.mohw.go.kr/') #웹페이지 url 끌어오기
source_corona = BeautifulSoup(webpage, 'html.parser',from_encoding='utf-8') #html.parser 이용하여 html 구조 가져오기

In [19]:
# source_corona

In [20]:
#웹페이지 개발자 도구 (윈도우는 f12, 맥은 사파리 설정)
#원하는 부분의 <A class =  B> 
#변수 = 위의 source.findAll( A, {'class':'B'})

div = source_corona.findAll('span',{'class': 'data'}) 

In [21]:
#html의 구조 get
div

[<span class="data">833</span>, <span class="data">37</span>]

In [10]:
#여기서 필요한 텍스트만 뽑아볼 것임
for a in div:
    print(a.get_text().strip())

833
37


In [11]:
#리스트화 시켜서 프린트 할 때 편하게...
list = []
for a in div:
    list.append(a.get_text())

In [12]:
list

['833', '37']

In [13]:
print('>> #오늘의 #국내 #코로나19 #현황 \n')
print('국내 유입:' ,list[0], '해외유입:' , list[1])

>> #오늘의 #국내 #코로나19 #현황 

국내 유입: 833 해외유입: 37


*여러 페이지에 있는 리뷰 크롤링하고, 파일로 저장하기*
========

> ### 1. 제목, 평점, 리뷰 들어있는 csv 만들어보기 
>>#### 네이버 평점, 리뷰 칸 속 평점.리뷰 들어가보면 여러개의 리뷰들이 나옴.
>### 2. html의 구조를 살펴보면
>>#### table.list_netizen > tbody > tr > td.title 속에 내용 다 있음.
>>>#### 제목 => a.movie.color_b
>>>#### 평점 => div.list_netizen_score > em
>>>#### 리뷰 => br 하위의 텍스트를 반환해야하는 것. => br.next_sibling 사용.

In [23]:
import requests #파이썬에서 http 요청 보내는 라이브러리 
from urllib import parse
from bs4 import BeautifulSoup #html 구조에서 간딴히 정보 빼내기 위해 사용하는 라이브러리
url = 'https://movie.naver.com/movie/point/af/list.nhn?&page=1'
res = requests.get(url)

soup = BeautifulSoup(res.text,'lxml')
tds = soup.select('table.list_netizen > tbody > tr > td.title')
print(len(tds))


10


In [24]:
tds

[<td class="title">
 <a class="movie color_b" href="?st=mcode&amp;sword=66520&amp;target=after">터미네이터: 미래전쟁의 시작</a>
 <div class="list_netizen_score">
 <span class="st_off"><span class="st_on" style="width:80%">별점 - 총 10점 중</span></span><em>8</em>
 </div>
 <br/>터미네이터 시리즈 중에선 그래도 상위권 
 			
 			
 			
 				
 				
 				
 				<a class="report" href="#" onclick="report('dxma****', 'fLot3Z2az3HLHUN7PJZthEBZq/nVMFeP/BHFK4a4teY=', '터미네이터 시리즈 중에선 그래도 상위권', '17329908', 'point_after');" style="color:#8F8F8F" title="새 창">신고</a>
 </td>,
 <td class="title">
 <a class="movie color_b" href="?st=mcode&amp;sword=199393&amp;target=after">조제</a>
 <div class="list_netizen_score">
 <span class="st_off"><span class="st_on" style="width:60%">별점 - 총 10점 중</span></span><em>6</em>
 </div>
 <br/>조제가 더 이상 기대지않고 혼자서도 일어서는 과정 
 			
 			
 			
 				
 				
 				
 				<a class="report" href="#" onclick="report('sseu****', '7lZ52U0tGmbnWuYl/HkYF55JoSyxaA3PQ+Qd72utXLg=', '조제가 더 이상 기대지않고 혼자서도 일어서는 과정 ', '17329907', 'point_af

In [25]:
tds[0]

<td class="title">
<a class="movie color_b" href="?st=mcode&amp;sword=66520&amp;target=after">터미네이터: 미래전쟁의 시작</a>
<div class="list_netizen_score">
<span class="st_off"><span class="st_on" style="width:80%">별점 - 총 10점 중</span></span><em>8</em>
</div>
<br/>터미네이터 시리즈 중에선 그래도 상위권 
			
			
			
				
				
				
				<a class="report" href="#" onclick="report('dxma****', 'fLot3Z2az3HLHUN7PJZthEBZq/nVMFeP/BHFK4a4teY=', '터미네이터 시리즈 중에선 그래도 상위권', '17329908', 'point_after');" style="color:#8F8F8F" title="새 창">신고</a>
</td>

In [26]:
comment_list=[]
for td in tds: 
    movie_title = td.find('a').text.strip()
#     link = td.select_one('a.movie.color_b').get('href')
#     link = parse.urljoin('https://movie.naver.com/movie/point/af/list.nhn?&page=2', link)
    score = td.select_one('div.list_netizen_score > em ').text.strip()
    comment = td.select_one('br').next_sibling.strip()
    comment_list.append((movie_title,score,comment))
    
    print(movie_title, score, comment, sep=" / ")

터미네이터: 미래전쟁의 시작 / 8 / 터미네이터 시리즈 중에선 그래도 상위권
조제 / 6 / 조제가 더 이상 기대지않고 혼자서도 일어서는 과정
디지몬 어드벤처 라스트 에볼루션 : 인연 / 9 / 어른이 됐다는걸 느끼게해주는영화 다음편도 나올것같음
차인표 / 1 / 차인표가 투자, 제작한줄
동주 / 10 / 인생영화! 여운이 남는 영화다
조제 / 1 / 저 영화보다 처음으로 잤어요 기억 안나는데 일어나서 오분정도 봤는데 영화 끝나서 뭐지? 했는데 앞에 분 울고 계시길래 빨리 나왔어요
수상한 그녀 / 10 / 짱재밌음 걍 보세요 내가 좋아하는 영화임
랭고 / 10 / 
차이나타운 / 6 / 스토리가 진부하고 개연성도 떨어져서보다가 갸우뚱하게됨그냥 비극적인 여성 느와르를 억지로 만든 느낌그래도 지루하진 않고, 김혜수 표정연기가 영화를조금은 살림
씽 / 8 / 음악으로 귀호강! 연말에 보기 좋은 황홀한 영화.두려움으로 사랑하는 것을 포기하지 말 것


In [29]:
import pandas as pd

df = pd.DataFrame(comment_list, columns=['제목','평점','리뷰'])
df
#df.to_csv('naver_comment.csv', encoding= 'utf-8', index=False)

,제목,평점,리뷰
0,터미네이터: 미래전쟁의 시작,8,터미네이터 시리즈 중에선 그래도 상위권
1,조제,6,조제가 더 이상 기대지않고 혼자서도 일어서는 과정
2,디지몬 어드벤처 라스트 에볼루션 : 인연,9,어른이 됐다는걸 느끼게해주는영화 다음편도 나올것같음
3,차인표,1,"차인표가 투자, 제작한줄"
4,동주,10,인생영화! 여운이 남는 영화다
5,조제,1,저 영화보다 처음으로 잤어요 기억 안나는데 일어나서 오분정도 봤는데 영화 끝나서 뭐...
6,수상한 그녀,10,짱재밌음 걍 보세요 내가 좋아하는 영화임
7,랭고,10,
8,차이나타운,6,스토리가 진부하고 개연성도 떨어져서보다가 갸우뚱하게됨그냥 비극적인 여성 느와르를 억...
9,씽,8,음악으로 귀호강! 연말에 보기 좋은 황홀한 영화.두려움으로 사랑하는 것을 포기하지 말 것


### 만약 두번째 페이지도 이어서 크롤링하고 싶다면? For 문을 사용하자!

https://movie.naver.com/movie/point/af/list.nhn?&page=1


https://movie.naver.com/movie/point/af/list.nhn?&page=2


이런식으로 페이지 뒷부분에 숫자의 변화가 생김.

In [30]:
base_url = 'https://movie.naver.com/movie/point/af/list.nhn?&page={}'
comment_list=[]

import time
import random

for page in range (10):
    url = base_url.format(page)
    res = requests.get(url)
    
    soup = BeautifulSoup(res.text,'lxml')
    tds = soup.select('table.list_netizen > tbody > tr > td.title')
    
    for td in tds: 
        movie_title = td.find('a').text.strip()
#         link = td.select_one('a.movie.color_b').get('href')
#         link = parse.urljoin('https://movie.naver.com/movie/point/af/list.nhn?&page=2', link)
        score = td.select_one('div.list_netizen_score > em ').text.strip()
        comment = td.select_one('br').next_sibling.strip()
        comment_list.append((movie_title,score, comment))

    interval = round(random.uniform(0.2,1),2)
    time.sleep(interval)

print('종료')

종료


In [31]:
df = pd.DataFrame(comment_list, columns=['제목','평점','리뷰'])
df.to_csv('naver_comment.csv', encoding= 'utf-8', index=False)

In [32]:
df

,제목,평점,리뷰
0,타이타닉 2,1,2.22점을 위한 노오력
1,리틀 포레스트,10,자극적인 맛이 아닌 깨끗하고 슴슴한 맛. 하지만 다양하고 깊다.
2,리틀 이블,6,연출과 편집은 깔끔했지만 중후반부 부터 똘끼가 떨어지며 이야기의 맥이 죽었다. 더 ...
3,콜,10,전종서 연기가 믿고 보는 영화...
4,월드 인베이젼,6,"네이버 평점보고 봤다가 후회막급, 전형적인 미국뽕영화 해병은 영원하다며 한명 영웅만..."
...,...,...,...
95,차인표,5,평점 알바가 정말로 존재한 다는 것을 알게됨..절대로 기대하고 보면 안됨. 10점 ...
96,해리 포터와 죽음의 성물 - 1부,10,
97,조제,8,"일본 원작과는 다른 느낌이지만, 애잔한 감동을 주네요.조제와 영석이 어디선가 행복하..."
98,헌터 킬러,9,긴장감 넘치는 연출 매끄러운 구성 스피디한 전개 거의 흠이 없음.
